<a href="https://colab.research.google.com/github/axel-sirota/cnn-visual-mastery/blob/main/Module3Demo1_VGG_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Transfer Learning from VGG-16 to solve cats vs dogs

In [ ]:
pip install tensorflow-datasets


Again we will load the dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

# Load Cats vs. Dogs dataset
(train_ds, validation_ds), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True,
)

And now asides resizing to vgg format, we will augment the dataset by flipping it left and right, up and down and changing brightness

In [ ]:

# Preprocess and augment the data
def preprocess(image, label):
    image = tf.image.resize(image, (150, 150))
    return image, label

def augment(image, label):
    # Data augmentation using TensorFlow ops
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.5)
    return image, label

# Apply preprocessing and augmentation to the dataset
train_ds = train_ds.map(preprocess).map(augment).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_ds = validation_ds.map(preprocess).map(augment).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)



Now we load VGG-16, freeze it and put it on top of our model, notice we don't need the Convolutions no more.

In [ ]:

# Load VGG16 model pre-trained on ImageNet data
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# # Freeze the layers
for layer in vgg_base.layers:
    layer.trainable = False

# Create a new model on top
model = Sequential([
    vgg_base,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.15),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


And we train, notice how much better it is

In [ ]:
# Train the model
callback = EarlyStopping(monitor='val_accuracy', patience=5)
history = model.fit(train_ds, epochs=100, validation_data=validation_ds, callbacks=[callback])

In [ ]:
def display_multiple_predictions(dataset, model, num_images=5):
    plt.figure(figsize=(15, 3))  # Set the figure size

    for i, (img, label) in enumerate(dataset.unbatch().take(num_images)):
        # Convert the label tensor to a numpy value
        actual_label = 'Dog' if label.numpy() == 1 else 'Cat'

        # Make a prediction
        predictions = model.predict(tf.expand_dims(img, 0))
        predicted_label = 'Dog' if predictions[0][0] > 0.5 else 'Cat'

        # Plotting
        plt.subplot(1, num_images, i+1)
        plt.imshow(img.numpy().astype("int32"))
        plt.title(f'A: {actual_label}\nP: {predicted_label}')
        plt.axis('off')

    plt.show()

In [ ]:
display_multiple_predictions(validation_ds, model)

Finally we can see on sample pictures how good the model is